In [1]:
import os
import time
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split

from util.eventvision import read_dataset, read_annotation
from util.BoundingBoxRegressionUtils import makeModel, parse_partial_dataset, parse_total_dataset, parse_random_dataset, tf_iou

POINTS = 500
INPUT_SHAPE = keras.Input(shape=(POINTS, 2, 1))

Event-based vision module imported


In [2]:
def makeBB(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    ax = plt.gca()
    return Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='red', facecolor='none', label="ground truth")

def getData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    
    points = np.column_stack((td.data.x[:POINTS], td.data.y[:POINTS]))
    
    return width, height, np.array(points).reshape(POINTS, 2, 1)

def getAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return minX, minY, maxX, maxY


In [3]:
def testDataTime():
    data_results = np.empty(20)
    
    for i in range(20):
        data_start_time = time.process_time() 

        X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        data_time = time.process_time() - data_start_time
        
        data_results[i] = data_time
    
    return np.mean(data_results)

# print("data avg", testDataTime())

In [4]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def test(model, test_data):
    predictions = model.predict(test_data[0])
    
    bb_scores = tf_iou(predictions[:, -4:], test_data[1][:, -4:])
        
    return np.mean(bb_scores)

def makeTestData():
    testData = []
    
    for i in range(10):
        testData.append(parse_random_dataset(getData, getAnnotation, True))
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_data = testData[i]

        testing_time = time.time()

        iou = test(model, test_data)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = makeModel(INPUT_SHAPE, 8)
    
    train_start_time = time.process_time()

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

Round  0
Training done!  19.1875
	For this mini round, time:  0.5539734363555908  iou  0.7328754
	For this mini round, time:  0.11403536796569824  iou  0.7251848
	For this mini round, time:  0.11896634101867676  iou  0.7340576
	For this mini round, time:  0.11502456665039062  iou  0.7331167
	For this mini round, time:  0.11997461318969727  iou  0.732649
	For this mini round, time:  0.11903023719787598  iou  0.7339872
	For this mini round, time:  0.11597013473510742  iou  0.74599594
	For this mini round, time:  0.1180257797241211  iou  0.7380139
	For this mini round, time:  0.12297415733337402  iou  0.7442063
	For this mini round, time:  0.11500096321105957  iou  0.72742796
Time on avg:  0.16129755973815918  IOU on avg:  0.7347514867782593
Round  1
Training done!  15.671875
	For this mini round, time:  0.17000389099121094  iou  0.6876162
	For this mini round, time:  0.11302661895751953  iou  0.6771315
	For this mini round, time:  0.11296963691711426  iou  0.68621135
	For this mini round

Training done!  15.046875
	For this mini round, time:  0.17800045013427734  iou  0.705042
	For this mini round, time:  0.12100052833557129  iou  0.6884802
	For this mini round, time:  0.11299896240234375  iou  0.6931332
	For this mini round, time:  0.11900067329406738  iou  0.6938422
	For this mini round, time:  0.11300015449523926  iou  0.6914871
	For this mini round, time:  0.11599993705749512  iou  0.69332224
	For this mini round, time:  0.11500048637390137  iou  0.7094577
	For this mini round, time:  0.11999917030334473  iou  0.7034873
	For this mini round, time:  0.11600017547607422  iou  0.7123475
	For this mini round, time:  0.11800026893615723  iou  0.69414467
Time on avg:  0.12290008068084717  IOU on avg:  0.6984744012355805
Round  12
Training done!  15.4375
	For this mini round, time:  0.17499971389770508  iou  0.7070196
	For this mini round, time:  0.11800026893615723  iou  0.69505644
	For this mini round, time:  0.11499953269958496  iou  0.7049215
	For this mini round, time

(array([19.1875  , 15.671875, 15.21875 , 15.171875, 15.484375, 15.15625 ,
        15.1875  , 15.65625 , 14.875   , 15.34375 , 15.59375 , 15.046875,
        15.4375  , 15.15625 , 15.921875, 15.671875, 15.515625, 15.5625  ,
        15.625   , 15.09375 ]),
 array([0.16129756, 0.12250006, 0.1223    , 0.12180259, 0.12260001,
        0.12159994, 0.12370055, 0.12400115, 0.14190209, 0.12939982,
        0.12159986, 0.12290008, 0.12339988, 0.12160006, 0.12339993,
        0.14799993, 0.12469981, 0.12279992, 0.12399998, 0.12230017]),
 array([0.73475149, 0.68642159, 0.73487067, 0.70551707, 0.72520822,
        0.68283122, 0.72845711, 0.74225891, 0.6980789 , 0.74682035,
        0.73541916, 0.6984744 , 0.70614563, 0.74592499, 0.70885661,
        0.68404802, 0.71171778, 0.67169251, 0.6890892 , 0.72307531]))

In [ ]:
def trainModel():
    model = makeModel(INPUT_SHAPE, 8)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=40,
        epochs=40,
        verbose=0)
    
    return history, model

history, model = trainModel()

In [6]:
w, h, points = getData(os.getcwd() + r'\..\Caltech101\airplanes\image_0002.bin')

inputPoints = points.reshape([1, POINTS, 2, 1])
preds = model.predict(inputPoints)[0]

minX = preds[4] * w
minY = preds[5] * h
maxX = preds[6] * w
maxY = preds[7] * h 


fig = plt.figure(figsize=(20,10))
plt.scatter(points[:,0], points[:,1])

ax = plt.gca()
ax.invert_yaxis()
ax.add_patch(makeBB(os.getcwd() + r'\..\Caltech101_annotations\airplanes\annotation_0002.bin'))
ax.add_patch(Rectangle((minX, minY), maxX - minX, maxY - minY, edgecolor='green', facecolor='none', label="predicted"))
ax.legend(loc="upper left", prop={'size': 15})

NameError: name 'model' is not defined

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

print(np.mean(history.history['val_loss']))
print(np.mean(history.history['val_accuracy']))